# Lang2LTL version 2

## Prerequisite: Import libraries 

In [ ]:
import pandas as pd
import json
import random
import os

from srer import srer

## 1. Generate natural language commands/utterances

### Load samples for utterances

In [ ]:
# -- load samples of LTL utterances from a CSV file using Pandas library:
ltl_samples = pd.read_csv(open('data/symbolic_batch12_noperm_pun.csv', 'r'))
display(ltl_samples[:10])

city_jsons = os.listdir('osm/')
selected_city = random.choice(city_jsons)
display(selected_city)

# -- load a city's landmarks from a JSON file (obtained from OpenStreetMap):
landmarks = json.load(open(f'osm/{selected_city}', 'r', encoding='utf8'))
display(list(landmarks.keys()))

# NOTE: several prepositions are taken from Kaiyu's paper (https://h2r.cs.brown.edu/wp-content/uploads/zheng2021spatial.pdf)
# -- each key is the phrase describing the spatial relation
# -- a key's corresponding value indicates the number of arguments it would take
spatial_relations = [
    {'between': 2},
    {'in front of': 1},
    {'behind': 1},
    {'next to': 1},
    {'adjacent to': 1},
    {'opposite': 1},
    {'near': 1},
    {'left of': 1},
    {'right of': 1},
    {'at': 1},
    {'north of': 1},
    {'south of': 1},
    {'east of': 1},
    {'west of': 1},
]

# -- list of landmarks in the city
landmarks = list(landmarks.keys())

# -- list of objects to use as reference objects:
generic_city_objects = [
    'bench', 'statue', 'stairs', 'store', 'door', 'bus stop', 'gas station', 'coffee shop',
    'bicycle rack', 'parking meter', 'parking spot', 'parking space', 'stop sign',
]


In [ ]:
# NOTE: this is Spot experiment specific content:

selected_city = 'blackstone.json'

# -- load a city's landmarks from a JSON file (obtained from OpenStreetMap):
landmarks = json.load(open(f'osm/{selected_city}', 'r', encoding='utf8'))
all_landmarks = list(landmarks.keys())

landmarks_to_use = ['Wildflour', 'Garden Grille Cafe']
for L in all_landmarks:
    if L not in landmarks_to_use:
        landmarks.pop(L)

# -- transforming specific names of large landmarks into generic categorical names:
landmarks['bakery'] = landmarks.pop('Wildflour')
landmarks['restaurant'] = landmarks.pop('Garden Grille Cafe')

# NOTE: below are the objects found in experiment location:
generic_city_objects = ['bicycle rack', 'chair', 'car']

landmarks = list(landmarks.keys())

### Define function for generating utterances

In [ ]:
def generate_synthetic_command(params, num_utterances=10, min_props=2, force_sre=False):
    examples, landmarks, city_objects, spatial_relations = params['samples'], params[
        'landmarks'], params['city_objects'], params['spatial_relations']

    list_utterances = []
    used_templates = []

    count = 0

    while count < num_utterances:
        # -- use the eval function to get the list of props for a randomly selected row from the set of LTL blueprints:
        random_sample = random.randint(1, len(examples))

        ltl_sample = examples.iloc[random_sample-1]
        ltl_propositions = eval(ltl_sample['props'])

        if len(ltl_propositions) < min_props:
            continue

        count += 1

        # display(ltl_sample)

        ltl_blueprint = ltl_sample['utterance']

        if not ltl_blueprint.endswith('.'):
            ltl_blueprint += '.'

        # -- save all original templates for matching them to reverse-engineered result later:
        used_templates.append(str(ltl_blueprint))

        # -- add a full-stop at the beginning and end of the sentence for easier tokenization:
        if not ltl_blueprint.startswith('.'):
            ltl_blueprint = '.' + ltl_blueprint

        for x in range(len(ltl_propositions)):

            new_entity = None

            # -- flip a coin to see if we will use a landmark or an object (potentially) near to the landmark:
            use_spatial_rel = random.randint(1, 2)

            # NOTE: if force_sre is switched to true, then it will generate propositions with all spatial referring expressions

            if use_spatial_rel > 1 or force_sre:
                # -- we will randomly select a spatial relation, breaking it down to its phrase and number of args:
                spatial_rel = random.choice(spatial_relations)

                # -- this is the spatial relation phrase/expression
                spatial_key = list(spatial_rel.keys())[-1]
                # -- this is the number of arguments it accepts
                spatial_args = spatial_rel[spatial_key]

                # NOTE: so far, we only have spatial relations with either 1 or 2 arguments.
                # -- are there possibly some with more?
                if spatial_args == 2:
                    # -- this is if we have a spatial relation with 2 arguments (e.g., between):

                    # -- randomly select 2 landmarks without replacement with which we will make a SRE:
                    landmark_samples = random.sample(landmarks, 2)

                    new_entity = f' the {random.choice(city_objects)} {spatial_key} {landmark_samples[0]} and {landmark_samples[1]}'
                elif spatial_args == 1:
                    # -- this is if we have a regular spatial relation with a single argument:

                    # -- randomly select only a single landmark:
                    new_entity = f' the {random.choice(city_objects)} {spatial_key} {random.choice(landmarks)}'
            else:
                # -- in this case, we will just select a landmark from the entire set in the city:
                new_entity = f' {random.choice(landmarks)}'

            # NOTE: to do replacement of the lifted proposition with the generated one, we need to account for
            # different ways it would be written preceded by a whitespace character, i.e., ' a ', ' a,', ' a.'

            # -- we will replace the proposition in the lifted expression with the grounded entity:
            props_to_replace = [(f' {ltl_propositions[x]}' + y) for y in [' ', '.', ',']] + [
                ('.' + f'{ltl_propositions[x]} '), ('.' + f'{ltl_propositions[x]},')]
            for prop in props_to_replace:
                # -- only replace if it was found:
                if prop in ltl_blueprint:
                    ltl_blueprint = ltl_blueprint.replace(
                        prop, new_entity + prop[-1])

            # NOTE: some utterances will be missing some propositions

        list_utterances.append(ltl_blueprint[1:])

    return used_templates, list_utterances


In [ ]:
generation_params = {
	'samples': ltl_samples,
	'landmarks': landmarks,
	'city_objects': generic_city_objects,
	'spatial_relations': spatial_relations
}

num_commands = 4

# -- forcing minimum number of propositions to be 5, forcing all SREs for propositions:
used_templates, utterances = generate_synthetic_command(generation_params, num_utterances=num_commands, min_props=2, force_sre=True)

for N in range(num_commands):
	print(f'{utterances[N]}\n')

## 2. Prompt LLM for spatial referring expressions

In [ ]:
count = 0

for command in utterances:
    raw_output, parsed_output = srer(command)

    parsed_output['lifted_gtr'] = used_templates[count]

    count += 1

    print(f'Command: {command}\n{json.dumps(parsed_output,indent=2)}')
    # display(parsed_output['command'])
    print(f"Groundtruth matched?: {parsed_output['lifted_llm'] == parsed_output['lifted_gtr']}" )
    print(f"-> LLM Lifted Translation: {parsed_output['lifted_llm']}")
    print(f"-> Groundtruth Lifted Translation: {parsed_output['lifted_gtr']}")


In [ ]:
utterances = list(filter(None, [X.strip() for X in open('blackstone_commands.txt', 'r').readlines()]))

print(utterances)

count = 0

all_llm_output = []

for command in utterances:
    raw_output, parsed_output = referring_exp_recognition(command)

    # parsed_output['lifted_gtr'] = used_templates[count]

    count += 1

    print(f'Command: {command}\n{json.dumps(parsed_output,indent=2)}')
    # display(parsed_output['command'])

    all_llm_output.append(parsed_output)

json.dump(all_llm_output, open('blackstone_srer_output.json', 'w'))